In [ ]:
%cd ..

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import glob, os

In [ ]:
df_items = pd.read_csv("data/items.csv")

base = "/Users/ivallesp/backup_tardigrade/cfavorita_results/"

paths = glob.glob(os.path.join(base, "results*.csv"))

In [ ]:
paths

## Sparsity

In [ ]:
file = paths[0]

In [ ]:
df = pd.read_csv(paths[0])

df.target.map(np.expm1).value_counts()[0]/df.target.map(np.expm1).value_counts().sum()

In [ ]:
df = pd.read_csv(paths[1])

df.target.map(np.expm1).value_counts()[0]/df.target.map(np.expm1).value_counts().sum()

In [ ]:
df = pd.read_csv(paths[2])

df.target.map(np.expm1).value_counts()[0]/df.target.map(np.expm1).value_counts().sum()

In [ ]:
(0.5246858950014468+0.5198615745261863+0.5260679456922743)/3

## Results

###  Table

In [ ]:
for path in reversed(sorted(paths)):
    print(os.path.split(path)[1])

    df = pd.read_csv(path)

    df = df.merge(df_items)
    
    df.perishable = (df.perishable*.25+1)


    from sklearn.metrics import mean_squared_error

    col = "forecast_transformer_655321"

    cols = df.columns[df.columns.str.contains("forecast")]
    rmsle = []
    rmswle = []
    male = []

    for col in cols:
        rmsle.append(np.sqrt(np.mean((df.target - df[col])**2)))
        rmswle.append(np.sqrt((df.perishable*(df.target - df[col])**2).sum()/df.perishable.sum()))
        male.append(np.mean(np.abs(df.target - df[col])))
    print("$", round(np.mean(rmsle), 4), "\\pm", round(np.std(rmsle), 4), "$", " & ",
    "$", round(np.mean(rmswle), 4), "\\pm", round(np.std(rmswle), 4), "$", " & ",
    "$", round(np.mean(male), 4), "\\pm", round(np.std(male), 4), "$")

In [ ]:
# Mean
for path in reversed(sorted(paths)):
    print(os.path.split(path)[1])

    df = pd.read_csv(path)

    df = df.merge(df_items)

    df.perishable = (df.perishable*.25+1)


    from sklearn.metrics import mean_squared_error

    col = "forecast_transformer_655321"

    cols = df.columns[df.columns.str.contains("forecast")]
    rmsle = []
    rmswle = []
    male = []

    for col in cols:
        rmsle.append(np.sqrt(np.mean((df.target - df.target.mean())**2)))
        rmswle.append(np.sqrt((df.perishable*(df.target - df.target.mean())**2).sum()/df.perishable.sum()))
        male.append(np.mean(np.abs(df.target - df.target.mean())))
    print("$", round(np.mean(rmsle), 4), "\\pm", round(np.std(rmsle), 4), "$", " & ",
    "$", round(np.mean(rmswle), 4), "\\pm", round(np.std(rmswle), 4), "$", " & ",
    "$", round(np.mean(male), 4), "\\pm", round(np.std(male), 4), "$")

In [ ]:
np.random.seed(655321)
# Random
for path in reversed(sorted(paths)):
    print(os.path.split(path)[1])

    df = pd.read_csv(path)

    df = df.merge(df_items)

    df.perishable = (df.perishable*.25+1)


    from sklearn.metrics import mean_squared_error

    col = "forecast_transformer_655321"

    cols = df.columns[df.columns.str.contains("forecast")]
    rmsle = []
    rmswle = []
    male = []

    for col in cols:
        s = df.target.sample(frac=1).values
        rmsle.append(np.sqrt(np.mean((df.target.values - s)**2)))
        rmswle.append(np.sqrt((df.perishable*(df.target.values - s)**2).sum()/df.perishable.sum()))
        male.append(np.mean(np.abs(df.target.values - s)))
    print("$", round(np.mean(rmsle), 4), "\\pm", round(np.std(rmsle), 4), "$", " & ",
    "$", round(np.mean(rmswle), 4), "\\pm", round(np.std(rmswle), 4), "$", " & ",
    "$", round(np.mean(male), 4), "\\pm", round(np.std(male), 4), "$")

### Daily

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
list(filter(lambda x: "lag_1" in x, reversed(sorted(paths))))

In [ ]:
plt.figure(figsize=(12, 3))
names = []
for path in filter(lambda x: "lag_1" in x, reversed(sorted(paths))):
    if "trim" in path:
        name = "s2s_trim"
    elif "transformer" in path:
        name = "transformer"
    else:
        name = "s2s"
    names.append(name)
    df = pd.read_csv(path)
    df["date"] = df.day.astype(str) + "-" + df.month.astype(str) + "-" + df.year.astype(str)
    df["date"] = pd.to_datetime(df.date, format="%d-%m-%Y")
    cols = df.columns[df.columns.str.contains("forecast")]

    results = []
    for col in cols:
        results.append((df.groupby("date")
         .apply(lambda d: np.sqrt(np.mean((d.target.values - d[col].values)**2)))))

    results = pd.concat(results)
    results_mu = results.groupby(results.index).mean()
    results_sigma = results.groupby(results.index).std()
    plt.errorbar(range(len(results_mu.index)), results_mu.values, yerr=results_sigma.values, capsize=3)
plt.xticks(range(len(results_mu.index)), results_mu.index.astype(str), rotation=50)
plt.grid()
plt.legend(names)
plt.ylabel("RMSLE")
plt.tight_layout()
plt.savefig("lag1_daily_error.eps")


In [ ]:
plt.figure(figsize=(12, 3))
names = []
for path in filter(lambda x: "lag_2" in x, reversed(sorted(paths))):
    if "trim" in path:
        name = "s2s_trim"
    elif "transformer" in path:
        name = "transformer"
    else:
        name = "s2s"
    names.append(name)
    df = pd.read_csv(path)
    df["date"] = df.day.astype(str) + "-" + df.month.astype(str) + "-" + df.year.astype(str)
    df["date"] = pd.to_datetime(df.date, format="%d-%m-%Y")
    cols = df.columns[df.columns.str.contains("forecast")]

    results = []
    for col in cols:
        results.append((df.groupby("date")
         .apply(lambda d: np.sqrt(np.mean((d.target.values - d[col].values)**2)))))

    results = pd.concat(results)
    results_mu = results.groupby(results.index).mean()
    results_sigma = results.groupby(results.index).std()
    plt.errorbar(range(len(results_mu.index)), results_mu.values, yerr=results_sigma.values, capsize=3)
plt.xticks(range(len(results_mu.index)), results_mu.index.astype(str), rotation=50)
plt.grid()
plt.legend(names)
plt.ylabel("RMSLE")
plt.tight_layout()
plt.savefig("lag2_daily_error.eps")


In [ ]:
plt.figure(figsize=(12, 3))
names = []
for path in filter(lambda x: "lag_3" in x, reversed(sorted(paths))):
    if "trim" in path:
        name = "s2s_trim"
    elif "transformer" in path:
        name = "transformer"
    else:
        name = "s2s"
    names.append(name)
    df = pd.read_csv(path)
    df["date"] = df.day.astype(str) + "-" + df.month.astype(str) + "-" + df.year.astype(str)
    df["date"] = pd.to_datetime(df.date, format="%d-%m-%Y")
    cols = df.columns[df.columns.str.contains("forecast")]

    results = []
    for col in cols:
        results.append((df.groupby("date")
         .apply(lambda d: np.sqrt(np.mean((d.target.values - d[col].values)**2)))))

    results = pd.concat(results)
    results_mu = results.groupby(results.index).mean()
    results_sigma = results.groupby(results.index).std()
    plt.errorbar(range(len(results_mu.index)), results_mu.values, yerr=results_sigma.values, capsize=3)
plt.xticks(range(len(results_mu.index)), results_mu.index.astype(str), rotation=50)
plt.grid()
plt.legend(names)
plt.ylabel("RMSLE")
plt.tight_layout()
plt.savefig("lag3_daily_error.eps")


In [ ]:
plt.figure(figsize=(12, 3))
names = []
for path in filter(lambda x: "lag_3" in x, reversed(sorted(paths))):
    if "trim" in path:
        name = "s2s_trim"
    elif "transformer" in path:
        name = "transformer"
    else:
        name = "s2s"
    names.append(name)
    df = pd.read_csv(path)
    df["date"] = df.day.astype(str) + "-" + df.month.astype(str) + "-" + df.year.astype(str)
    df["date"] = pd.to_datetime(df.date, format="%d-%m-%Y")
    cols = df.columns[df.columns.str.contains("forecast")]

    results = []
    for col in cols:
        results.append((df.groupby("date")
         .apply(lambda d: np.sqrt(np.mean((d.target.values - d[col].values)**2)))))


    plt.errorbar(range(len(results_mu.index)), results_mu.values, yerr=results_sigma.values, capsize=3)
plt.xticks(range(len(results_mu.index)), results_mu.index.astype(str), rotation=50)
plt.grid()
plt.legend(names)
plt.ylabel("RMSLE")
plt.tight_layout()
plt.savefig("lag3_daily_error.eps")


### SAMPLES 

In [ ]:
from tqdm import tqdm

In [ ]:
dfs = []
for path in tqdm(paths):
    if "trim" in path:
        name = "s2s_trim"
    elif "transformer" in path:
        name = "transformer"
    else:
        name = "s2s"
    
    if "lag_3" in path:
        lag = 3
    elif "lag_2" in path:
        lag = 2
    else:
        lag = 1
        
    df = pd.read_csv(path)
    df["date"] = df.day.astype(str) + "-" + df.month.astype(str) + "-" + df.year.astype(str)
    df["date"] = pd.to_datetime(df.date, format="%d-%m-%Y")
    cols = df.columns[df.columns.str.contains("forecast")]
    newcols = ["forecast_"+str(i) for i in range(len(cols))]
    df = df.rename(columns = dict(zip(cols, newcols)))
    df["model"] = name
    df["lag"] = lag
    dfs.append(df)

In [ ]:
df_unified = pd.concat(dfs)

In [ ]:
cases = df.groupby(["store_nbr", "item_nbr"]).target.sum()
cases = cases[cases.rank()>int(cases.shape[0]*0.8)]
np.random.seed(655334)
cases = (cases[cases>0].reset_index().sample(5))[["store_nbr", "item_nbr"]]
store_nbrs, item_nbrs = zip(*zip(cases.store_nbr.values, cases.item_nbr.values))

In [ ]:
item_nbrs = [862213, 996606, 1473409, 1239906, 414750]
store_nbrs = [3, 47, 42, 19, 48]

In [ ]:
for i in range(5):
    store_nbr = store_nbrs[i]
    item_nbr = item_nbrs[i]
    m = -1
    target_plotted=False
    if i<4:
        plt.figure(figsize=(12, 2))
    else:
        plt.figure(figsize=(12, 2.65))
    
    for mdl in tqdm(df_unified.model.unique()):
        df_ts = df_unified.loc[lambda d:(d.store_nbr == store_nbr) & (d.item_nbr == item_nbr) & (d.model == mdl)]

        mu = df_ts[df_ts.columns[df_ts.columns.str.contains("forecast")]].mean(axis=1)
        sigma = df_ts[df_ts.columns[df_ts.columns.str.contains("forecast")]].std(axis=1)
        target = df_ts.target
        x = df_ts.date

        if not target_plotted:
            target_plotted=True

            plt.plot(range(len(x)), target)

        plt.errorbar(range(len(x)), mu, yerr=sigma, capsize=3)
        m = max(m, max(target.max(), (mu+sigma).max()))
    plt.ylim(0,m)
    plt.xticks(range(len(x)), x.astype(str), rotation=50)
    plt.grid()
    plt.ylabel(f"Log Sales\nitem: {item_nbr}\nstore: {store_nbr}")
    if i<4:
        plt.gca().axes.set_xticklabels([])
    if i==0:
        plt.legend(["target"] + df_unified.model.unique().tolist(), loc="upper right")
    plt.tight_layout()
    plt.savefig(f"sample_{i}_log.eps")

In [ ]:
for i in range(5):
    store_nbr = store_nbrs[i]
    item_nbr = item_nbrs[i]
    m = -1
    target_plotted=False
    if i<4:
        plt.figure(figsize=(12, 2))
    else:
        plt.figure(figsize=(12, 2.65))

    for mdl in tqdm(df_unified.model.unique()):
        df_ts = df_unified.loc[lambda d:(d.store_nbr == store_nbr) & (d.item_nbr == item_nbr) & (d.model == mdl)]

        mu = df_ts[df_ts.columns[df_ts.columns.str.contains("forecast")]].apply(np.expm1).mean(axis=1).round()
        sigma = df_ts[df_ts.columns[df_ts.columns.str.contains("forecast")]].apply(np.expm1).std(axis=1)
        target = df_ts.target.map(np.expm1)
        x = df_ts.date

        if not target_plotted:
            target_plotted=True

            plt.plot(range(len(x)), target)

        plt.errorbar(range(len(x)), mu, yerr=sigma, capsize=3)
        m = max(m, max(target.max(), (mu+sigma).max()))
    plt.ylim(0,m)
    plt.xticks(range(len(x)), x.astype(str), rotation=50)
    plt.grid()
    plt.ylabel(f"Sales\nitem: {item_nbr}\nstore: {store_nbr}")
    if i<4:
        plt.gca().axes.set_xticklabels([])
    if i==0:
        plt.legend(["target"] + df_unified.model.unique().tolist(), loc="upper right")

    plt.tight_layout()
    plt.savefig(f"sample_{i}_lin.eps")